In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  
from IPython.display import HTML

In [2]:
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, precision_score, recall_score
import pandas
import numpy as np
import papermill as pm
import json
import matplotlib.pyplot as plt
import os
import uuid
from db import Result
import ast
import math
import scrapbook as sb
pandas.options.display.float_format = '{:,.3f}'.format

In [3]:
from evaluate import plot_learning_curve, plot_accuracy_curve, load, get_results, get_labels, transform_binary_probabilities, transform_binary_predictions, calculate_accuracy_loss, plot_confusion_matrix, plot_precision_recall, plot_roc_curve, calculate_pr_auc, calculate_confusion_matrix_stats, calculate_confusion_matrix, plot_precision_recall#, plot_tsne, plot_multiple_grad_cam
from data_gen import data
from run import characterize_data
from config import config

In [4]:
# T1 results
t1_results = Result.query.filter(Result.description == "t1_20210402").all()
#t1_results = Result.query.filter(Result.description == "t1_final_200").all()

# T2 results
t2_results = Result.query.filter(Result.description == "t2_20210402").all()
#t2_results = Result.query.filter(Result.description == "t2_final_run_200").all()

In [5]:
print(t1_results)
print(t2_results)

[<Result accuracy: 0.6439790725708008>, <Result accuracy: 0.6753926873207092>, <Result accuracy: 0.6596858501434326>, <Result accuracy: 0.6963350772857666>, <Result accuracy: 0.6910994648933411>, <Result accuracy: 0.6910994648933411>, <Result accuracy: 0.6753926873207092>, <Result accuracy: 0.6910994648933411>, <Result accuracy: 0.6806282997131348>, <Result accuracy: 0.6910994648933411>]
[<Result accuracy: 0.6649214625358582>, <Result accuracy: 0.6282722353935242>, <Result accuracy: 0.5863874554634094>, <Result accuracy: 0.6439790725708008>, <Result accuracy: 0.6544502377510071>, <Result accuracy: 0.6335078477859497>, <Result accuracy: 0.6492146849632263>, <Result accuracy: 0.6544502377510071>, <Result accuracy: 0.717277467250824>, <Result accuracy: 0.6806282997131348>]


In [6]:
train, validation, test, external = data(seed=uuid.UUID("84a64c17-fe3e-440c-aaaf-e1bd5b02576f"), label_form="outcome_pos", input_form="t1", train_shuffle=False, test_shuffle=False, validation_shuffle=False, train_augment=False, validation_augment=False, test_augment=False)
all_results_t1 = {}
for result in t1_results:
    MODEL = "{}/models/{}-{}.h5".format(config.OUTPUT, result.uuid, result.model)
    model = load(MODEL)
    results = get_results(model, validation)
    labels = get_labels(validation)
    stats = calculate_confusion_matrix_stats(labels, results)
    all_results_t1[str(result.uuid)] = {"sensitivity": stats["TPR"][1], "specificity": stats["TNR"][1], "roc_auc": roc_auc_score(labels,results)}

train, validation, test, external = data(seed=uuid.UUID("84a64c17-fe3e-440c-aaaf-e1bd5b02576f"), label_form="outcome_pos", input_form="t2", train_shuffle=False, test_shuffle=False, validation_shuffle=False, train_augment=False, validation_augment=False, test_augment=False)
all_results_t2 = {}
for result in t2_results:
    MODEL = "{}/models/{}-{}.h5".format(config.OUTPUT, result.uuid, result.model)
    model = load(MODEL)
    results = get_results(model, validation)
    labels = get_labels(validation)
    stats = calculate_confusion_matrix_stats(labels, results)
    all_results_t2[str(result.uuid)] = {"sensitivity": stats["TPR"][1], "specificity": stats["TNR"][1], "roc_auc": roc_auc_score(labels,results)}

  0%|          | 1/678 [00:00<01:23,  8.13it/s]

                outcome_pos  outcome_neg  outcome_3    age  sex  \
patientID                                                         
bone-brown-1              0            0          0 52.000    1   
bone-brown-10             0            0          0 52.000    0   
bone-brown-100            1            1          2 22.000    1   
bone-brown-101            0            0          0 88.000    0   
bone-brown-102            0            0          0 56.000    1   
...                     ...          ...        ...    ...  ...   
bone-penn-650             1            1          2 20.000    1   
bone-penn-651             0            0          0 16.000    1   
bone-penn-652             1            1          2  8.000    0   
bone-penn-653             1            1          2 16.000    1   
bone-penn-654             1            1          2  3.000    1   

                                                         location        sort  
patientID                                       

  0%|          | 2/678 [00:00<02:48,  4.02it/s]

bone-brown-101
I'm doing the thing.


  0%|          | 3/678 [00:00<04:03,  2.77it/s]

...

/home/ubuntu/miniconda3/envs/bone-mri/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


/home/ubuntu/miniconda3/envs/bone-mri/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


/home/ubuntu/miniconda3/envs/bone-mri/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


/home/ubuntu/miniconda3/envs/bone-mri/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


/home/ubuntu/miniconda3/envs/bone-mri/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [7]:
import pandas as pd
print("T1")
print(pd.DataFrame.from_dict(all_results_t1, orient='index'))
print("T2")
print(pd.DataFrame.from_dict(all_results_t2, orient='index'))

T1
                                      sensitivity  specificity  roc_auc
eb7238a6-8026-4beb-9614-7382c1a732e4        0.744        0.562    0.686
610fcfce-1b49-40ce-bf2a-adc6a6368dfe        0.547        0.781    0.704
1c4a0957-81ee-475d-b023-a2f07af39d4c        0.674        0.648    0.719
80a0d77f-ee08-4eb1-89f1-f106ca4fb2fe        0.721        0.676    0.743
0e7eda78-945a-4a0b-ba47-29c9a77414cf        0.570        0.790    0.764
b0377056-b7c9-4869-9f36-a8202eb59132        0.802        0.600    0.742
775b9b1a-dbeb-40ea-9d9e-43f3e92edffb        0.686        0.667    0.728
565fc28f-9ac2-40fa-af9f-9d35761e1735        0.581        0.781    0.725
761976c5-ce25-4ab6-8718-9b7df5cc5123        0.733        0.638    0.708
def3cd09-848f-4925-bde0-8929aad34a6c        0.791        0.610    0.751
T2
                                      sensitivity  specificity  roc_auc
5b608461-3ffe-4baa-8a52-07a1e04e33d2        0.488        0.810    0.730
61ede923-9484-42df-86e8-6821dd113562        0.709        0